In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
val=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
sub=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")

In [3]:
data=data.fillna(data.median())

/tmp/ipykernel_20/495656529.py:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data=data.fillna(data.median())


In [4]:
from sklearn.preprocessing import LabelEncoder

data['EJ'] = LabelEncoder().fit_transform(data['EJ'])
data=data.drop(['Id'], axis=1)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
val['EJ'] = LabelEncoder().fit_transform(val['EJ'])
val=val.drop(['Id'], axis=1)

In [6]:
x=data.drop(["Class"], axis=1)
y=data["Class"]

In [7]:
from sklearn.model_selection import train_test_split

#x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [8]:
import xgboost as xgb
dtrain = xgb.DMatrix(x, label=y)
#dtest = xgb.DMatrix(x_test, label=y_test)

In [9]:
#making brier scorer
from sklearn.metrics import make_scorer, brier_score_loss

brier_scorer = make_scorer(brier_score_loss, greater_is_better=False)

In [10]:
params = {
    'objective': 'binary:logistic',
'colsample_bytree': 0.3,
 'gamma': 0.1,
 'learning_rate': 0.3,
 'max_depth': 5}

In [11]:
model = xgb.train(params, dtrain, 50)

In [12]:
#y_pred = model.predict(dtest)

In [13]:
#brier_score = brier_score_loss(y_test, y_pred)
#print("Brier score:", brier_score)


In [14]:
#from sklearn.metrics import accuracy_score

# Convert probabilities to binary predictions


#y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]
#
#accuracy = accuracy_score(y_test, y_pred_binary)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [15]:
#from sklearn.metrics import confusion_matrix

# Assuming you have the true labels for the test set stored in `y_true`
# and the predicted labels in `y_pred`
#confusion = confusion_matrix(y_test, y_pred_binary)

# Extracting true negatives (TN), false positives (FP),
# false negatives (FN), and true positives (TP)
#TN = confusion[0, 0]
#FP = confusion[0, 1]
#FN = confusion[1, 0]
#TP = confusion[1, 1]

# Calculating false negative rate and false positive rate
#false_negative_rate = FN / float(FN + TP)
#false_positive_rate = FP / float(FP + TN)

# Print or use the false negative rate and false positive rate as needed
#print("False Negative Rate:", false_negative_rate)
#print("False Positive Rate:", false_positive_rate)

In [16]:
label= val.shape[0]
dval = xgb.DMatrix(val, np.zeros(label))

In [17]:
pred=model.predict(dval)


In [18]:
pred=pred.astype(float)
pred.resize(label,1)
pred.shape

(5, 1)

In [19]:
sub[['class_1']]=pred
sub[['class_0']]= 1-pred

In [20]:
sub.to_csv("submission.csv", index=False)

In [21]:
sub


,Id,class_0,class_1
0,00eed32682bb,0.911551,0.088449
1,010ebe33f668,0.911551,0.088449
2,02fa521e1838,0.911551,0.088449
3,040e15f562a2,0.911551,0.088449
4,046e85c7cc7f,0.911551,0.088449
